# 2023 CVE CPE Review 

In [1]:
%%capture
!mkdir -p jsondata
%cd jsondata
!rm *.json 
!rm *.zip 
!wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{2002..2023}.json.zip 
!unzip -o "*.zip" 
!wget https://raw.githubusercontent.com/CVEProject/cve-website/dev/src/assets/data/CNAsList.json

In [2]:

from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import calplot
import datetime
from datetime import date
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))


logging.getLogger('matplotlib.font_manager').disabled = True
warnings.filterwarnings("ignore")
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 10)

In [3]:
row_accumulator = []

for filename in glob.glob('nvdcve-1.1-*.json'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for cves in nvd_data['CVE_Items']:
            if cves['configurations']['nodes'] != []:
                if "children" in cves['configurations']['nodes'][0]:
                    for j in range(len(cves['configurations']['nodes'][0]['children'])):
                        for cpe in cves['configurations']['nodes'][0]['children'][j]['cpe_match']:
                            if cpe['vulnerable'] == True:
                                        cve = cves['cve']['CVE_data_meta']['ID']
                                        try:
                                            published_date = cves['publishedDate']
                                        except KeyError:
                                            published_date = 'Missing_Data'
                                        cpe_string = cpe['cpe23Uri']
                                        try:
                                            end = cpe['versionEndExcluding']
                                        except: 
                                            end = 'None'
                                        try:
                                            end2 = cpe['versionEndIncluding']
                                        except: 
                                            end2 = 'None' 
                                        try:
                                            start = cpe['versionStartExcluding']
                                        except: 
                                            start = 'None'
                                        try:
                                            start2 = cpe['versionStartIncluding']
                                        except: 
                                            start2 = 'None'     
                                        new_row = { 
                                        'CVE': cve,
                                        'Published': published_date,
                                        'CPE' : cpe_string,
                                        'StartI' : start,
                                        'StartE' : start2,
                                        'EndI' : end,
                                        'EndE' : end2 
                                        }
                                        row_accumulator.append(new_row)
        


nvd = pd.DataFrame(row_accumulator)  
nvd['Published'] = pd.to_datetime(nvd['Published'])
thisyear = ((nvd['Published'] > '2023-01-01') & (nvd['Published']  < '2024-01-01'))
nvd = nvd.loc[thisyear]
nvd = nvd.sort_values(by=['Published'])
nvd = nvd.reset_index(drop=True)
nvdcount = nvd['Published'].count()
nvdunique = nvd['Published'].nunique()
startdate = date(2023, 1, 1)
enddate  = date(2024, 1, 1)
numberofdays = enddate - startdate 
per_day = nvdcount/numberofdays.days

### CVEs With Most CPEs


In [4]:
nvd['CVE'].value_counts().head(20)

CVE-2023-44183    240
CVE-2023-36848    213
CVE-2023-36850    207
CVE-2023-20187    201
CVE-2023-20027    190
CVE-2023-36832    181
CVE-2023-28976    152
CVE-2023-28972    149
CVE-2023-22394    127
CVE-2023-1697     122
CVE-2023-22409    122
CVE-2023-28959    121
CVE-2023-22411    114
CVE-2023-22404    109
CVE-2023-22417    105
CVE-2023-22415    105
CVE-2023-28974    104
CVE-2023-28965    101
CVE-2023-36838    101
CVE-2023-20227     96
Name: CVE, dtype: int64

## Most Common CPEs

In [5]:
nvd['CPE'].value_counts().head(20)

cpe:2.3:o:google:android:12.0:*:*:*:*:*:*:*                                    547
cpe:2.3:o:google:android:11.0:*:*:*:*:*:*:*                                    451
cpe:2.3:o:google:android:13.0:*:*:*:*:*:*:*                                    385
cpe:2.3:o:google:android:10.0:*:*:*:*:*:*:*                                    287
cpe:2.3:o:google:android:11.0:-:*:*:*:*:*:*                                     99
cpe:2.3:a:adobe:acrobat_dc:*:*:*:*:continuous:*:*:*                             92
cpe:2.3:a:adobe:acrobat_reader_dc:*:*:*:*:continuous:*:*:*                      92
cpe:2.3:o:arubanetworks:arubaos:*:*:*:*:*:*:*:*                                 87
cpe:2.3:o:google:android:12.0:-:*:*:*:*:*:*                                     79
cpe:2.3:o:insteon:hub_firmware:1012:*:*:*:*:*:*:*                               76
cpe:2.3:a:nvidia:virtual_gpu:*:*:*:*:*:*:*:*                                    75
cpe:2.3:o:qualcomm:aqt1000_firmware:-:*:*:*:*:*:*:*                             71
cpe:

In [6]:
nvd['CPE'].value_counts().describe()

count    3119.000000
mean        4.645720
std        17.241405
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max       547.000000
Name: CPE, dtype: float64